In [1]:
import pandas as pd

data = pd.read_csv("/water_potability.csv")
print(data.head())


         ph    Hardness        Solids  Chloramines     Sulfate  Conductivity  \
0       NaN  204.890455  20791.318981     7.300212  368.516441    564.308654   
1  3.716080  129.422921  18630.057858     6.635246         NaN    592.885359   
2  8.099124  224.236259  19909.541732     9.275884         NaN    418.606213   
3  8.316766  214.373394  22018.417441     8.059332  356.886136    363.266516   
4  9.092223  181.101509  17978.986339     6.546600  310.135738    398.410813   

   Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       10.379783        86.990970   2.963135           0  
1       15.180013        56.329076   4.500656           0  
2       16.868637        66.420093   3.055934           0  
3       18.436524       100.341674   4.628771           0  
4       11.558279        31.997993   4.075075           0  


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier


In [4]:
# Handle missing values
data.fillna(data.mean(), inplace=True)

X = data.drop("Potability", axis=1)
y = data["Potability"]


In [5]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(
        n_estimators=100,
        random_state=42
    ))
])


In [6]:
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

scores = cross_val_score(
    pipeline,
    X,
    y,
    cv=cv,
    scoring='accuracy'
)


In [7]:
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

scores = cross_val_score(
    pipeline,
    X,
    y,
    cv=cv,
    scoring='accuracy'
)


In [8]:
print("Cross-Validation Accuracy Scores:", scores)
print("Average Accuracy:", scores.mean())
print("Standard Deviation:", scores.std())


Cross-Validation Accuracy Scores: [0.67987805 0.68244275 0.65954198 0.66717557 0.65648855]
Average Accuracy: 0.669105380748464
Standard Deviation: 0.010471894528639135


In [9]:
pipeline.fit(X, y)

sample = [[7.2, 180, 12000, 7, 300, 400, 10, 60, 3]]
prediction = pipeline.predict(sample)

print("Prediction:", "Safe" if prediction[0] == 1 else "Not Safe")


Prediction: Not Safe


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [10]:
!pip install flask ngrok joblib


In [11]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

# Load data
data = pd.read_csv("/water_potability.csv")
data.fillna(data.mean(), inplace=True)

X = data.drop("Potability", axis=1)
y = data["Potability"]

# Pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Train final model
pipeline.fit(X, y)

# Save model
joblib.dump(pipeline, "water_model.pkl")

print("Model saved successfully")


Model saved successfully


In [ ]:
from flask import Flask, render_template, request
import joblib
import numpy as np
import os

app = Flask(__name__)

model = joblib.load("water_model.pkl")

@app.route('/')
def home():
    return render_template("index.html")

@app.route('/predict', methods=['POST'])
def predict():
    features = [float(x) for x in request.form.values()]
    final_features = [np.array(features)]
    prediction = model.predict(final_features)

    result = "SAFE for Drinking" if prediction[0] == 1 else "NOT SAFE for Drinking"
    return render_template("index.html", prediction_text=result)

if __name__ == "__main__":
    port = int(os.environ.get("PORT", 5000))
    app.run(host="0.0.0.0", port=port)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [13]:
!mkdir templates


mkdir: cannot create directory ‘templates’: File exists


In [14]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>Water Quality Detection</title>
</head>
<body>
    <h2>Water Quality Detection System</h2>

    <form action="/predict" method="post">
        <input type="number" step="any" name="ph" placeholder="pH" required><br><br>
        <input type="number" step="any" name="Hardness" placeholder="Hardness" required><br><br>
        <input type="number" step="any" name="Solids" placeholder="Solids" required><br><br>
        <input type="number" step="any" name="Chloramines" placeholder="Chloramines" required><br><br>
        <input type="number" step="any" name="Sulfate" placeholder="Sulfate" required><br><br>
        <input type="number" step="any" name="Conductivity" placeholder="Conductivity" required><br><br>
        <input type="number" step="any" name="Organic_carbon" placeholder="Organic Carbon" required><br><br>
        <input type="number" step="any" name="Trihalomethanes" placeholder="Trihalomethanes" required><br><br>
        <input type="number" step="any" name="Turbidity" placeholder="Turbidity" required><br><br>

        <button type="submit">Check Water Quality</button>
    </form>

    <h3>{{ prediction_text }}</h3>
</body>
</html>


Overwriting templates/index.html


In [15]:
!pip install pyngrok


In [16]:
!ngrok config add-authtoken 387G1bMZKy1XziqG4lOrgWgBer2_5nGAaCBbPKZk1kgcb8dZo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print(public_url)


NgrokTunnel: "https://farsightedly-nonbeatific-audie.ngrok-free.dev" -> "http://localhost:5000"


In [18]:
app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Jan/2026 15:21:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2026 15:21:39] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [11/Jan/2026 15:22:52] "POST /predict HTTP/1.1" 200 -


In [1]:
from flask import Flask, render_template, request
import joblib
import numpy as np

app = Flask(__name__)
model = joblib.load("water_model.pkl")


In [2]:
@app.route('/')
def home():
    return render_template("index.html")

@app.route('/predict', methods=['POST'])
def predict():
    features = [float(x) for x in request.form.values()]
    final_features = [np.array(features)]
    prediction = model.predict(final_features)

    result = "SAFE for Drinking" if prediction[0] == 1 else "NOT SAFE for Drinking"
    return render_template("index.html", prediction_text=result)


In [3]:
app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [4]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print("Your website link:", public_url)



Your website link: NgrokTunnel: "https://farsightedly-nonbeatific-audie.ngrok-free.dev" -> "http://localhost:5000"


In [5]:
from pyngrok import ngrok

ngrok.kill()  # clears old tunnels

public_url = ngrok.connect(5000)
print("🌐 Your website link:", public_url)


🌐 Your website link: NgrokTunnel: "https://farsightedly-nonbeatific-audie.ngrok-free.dev" -> "http://localhost:5000"
